## EMA y Fibonacci

In [1]:
# @title
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime
from dateutil.relativedelta import relativedelta

# !pip install yfinance plotly --quiet


In [2]:
#@title Seleccione Acción y Medias Móviles Exponenciales{run:'auto'}
TICKER = "NVDA" #@param ["AAPL", "PLTR", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "TSM","BRK.B", "V", "JPM", "XOM", "LLY", "MRK", "UNH", "PG", "MA","CVX", "KO", "PEP", "COST", "TMO", "ORCL", "CSCO", "NKE", "VZ", "ASML", "TXN", "ABT", "TM", "SAP", "AMD", "NFLX", "NOW", "ADBE", "LVMUY", "BABA", "SHEL", "TMUS", "QCOM", "PFE", "SNY", "AZN", "TOT", "GSK", "RIO", "BHP", "MCD​"]
#ticker_symbol = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]
YEARS = 1 #@param {type:"integer"}
END   =  datetime.today().strftime("%Y-%m-%d")
START = (datetime.today() - relativedelta(years=YEARS)).strftime("%Y-%m-%d")
EMA1 = 10 #@param {type:"integer"}
EMA2  = 20 #@param {type:"integer"}
EMA_CORR   = 100 #@param {type:"integer"}
TREND_MA       = 200 #@param {type:"integer"}

In [3]:
# Confluencia MA (Golden/Death Cross) + Fibonacci, señales y gráfico interactivo
# - MA1 y MA2 son parametrizables (por defecto 10 y 20)
# - Filtro de tendencia con TREND_MA (por defecto 200)
# - Señales (score >= SCORE_THRESHOLD) y zonas de Fib 38.2–61.8%


In [4]:
# =========================
# Parámetros (modifica aquí)
# =========================
TICKER         = "NVDA"
START          = "2023-01-01"
END            = None  # None = hoy
MA1            = 20    # Rápida (Golden/Death cross con MA2)
MA2            = 50    # Lenta  (Golden/Death cross con MA1)
TREND_MA       = 100   # Filtro de tendencia (pendiente + lado del precio)
CONFIRM_EMA    = MA1   # EMA de confirmación (por defecto, igual a MA1)
W_SWING        = 10    # Ventana de pivotes para swings
SCORE_THRESHOLD= 6     # Umbral para aceptar señales
LAST_N         = 300   # Velas a mostrar (None = todo)
SHOW_EXT       = True  # Mostrar extensiones 1.272 y 1.618
SHOW_SIGNALS   = False   # False = se ocultan por defecto (clic en la leyenda para mostrarlas)
SIGNAL_OPACITY = 0.25    # 0.10–0.35 = muy tenue
SIG_COLOR      = "rgba(207,181,59,0.35)"  # dorado tenue (old gold con alfa)
UP_COLOR   = "#FF1500"  # orange
DOWN_COLOR = "#0080FF"  # blue

In [5]:

# =========================
# Utilidades base
# =========================
def sma(s, n): return s.rolling(n).mean()
def ema(s, n): return s.ewm(span=n, adjust=False).mean()

def download_prices(tickers, start, end=None, auto_adjust=True):
    """
    Descarga precios con yfinance.
    Aplana MultiIndex y devuelve columnas simples: Open, High, Low, Close, Volume (si existen).
    Acepta str o lista; si es lista, toma el primer ticker para un OHLC plano.
    """
    if end is None:
        end = pd.Timestamp.today().strftime('%Y-%m-%d')

    df = yf.download(tickers, start=start, end=end, auto_adjust=auto_adjust, progress=False)
    if df.empty:
        return df

    # Aplana columnas si vienen en MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        if isinstance(tickers, (list, tuple, set)):
            tick_list = list(tickers)
            tick = tick_list[0]
        else:
            tick = tickers
        if tick in df.columns.get_level_values(-1):
            df = df.xs(tick, axis=1, level=-1)  # -> columnas simples
        else:
            df.columns = df.columns.get_level_values(0)

    # Conservar columnas típicas si existen
    keep = [c for c in ['Open','High','Low','Close','Adj Close','Volume'] if c in df.columns]
    df = df[keep].copy()

    # Asegurar numéricos
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Quitar filas totalmente vacías en OHLC
    if set(['Open','High','Low','Close']).issubset(df.columns):
        df = df.dropna(subset=['Open','High','Low','Close'], how='all')

    return df

def atr(df, n=14):
    h,l,c = df['High'], df['Low'], df['Close']
    tr = np.maximum(h - l, np.maximum((h - c.shift()).abs(), (l - c.shift()).abs()))
    return tr.rolling(n).mean()

def slope(series, n=5):
    diff = series - series.shift(n)
    with np.errstate(divide='ignore', invalid='ignore'):
        out = (diff / (n * series)).replace([np.inf, -np.inf], np.nan)
    return out.fillna(0)

def pivots(df, w=10):
    highs = df['High']; lows = df['Low']
    is_high = highs.eq(highs.rolling(window=2*w+1, center=True).max())
    is_low  = lows.eq(lows.rolling(window=2*w+1, center=True).min())
    df['swing_high'] = np.where(is_high, highs, np.nan)
    df['swing_low']  = np.where(is_low,  lows,  np.nan)
    return df

def last_swing_levels(df, trend='up'):
    if trend=='up':
        hi_idx = df['swing_high'].last_valid_index()
        lo_idx = df['swing_low'].loc[:hi_idx].last_valid_index() if hi_idx else None
        if not (hi_idx and lo_idx and lo_idx < hi_idx): return None
        lo, hi = df.at[lo_idx,'Low'], df.at[hi_idx,'High']
    else:
        lo_idx = df['swing_low'].last_valid_index()
        hi_idx = df['swing_high'].loc[:lo_idx].last_valid_index() if lo_idx else None
        if not (hi_idx and lo_idx and hi_idx < lo_idx): return None
        hi, lo = df.at[hi_idx,'High'], df.at[lo_idx,'Low']

    rng = hi - lo
    if pd.isna(rng) or rng <= 0: return None

    return {
        '0.382': hi - 0.382*rng if trend=='up' else lo + 0.382*rng,
        '0.5'  : hi - 0.5  *rng if trend=='up' else lo + 0.5  *rng,
        '0.618': hi - 0.618*rng if trend=='up' else lo + 0.618*rng,
        '0.786': hi - 0.786*rng if trend=='up' else lo + 0.786*rng,
        'ext_1.272': hi + 0.272*rng if trend=='up' else lo - 0.272*rng,
        'ext_1.618': hi + 0.618*rng if trend=='up' else lo - 0.618*rng,
    }

def fib_zone_hit(close, levels, atr_val, mult=0.5):
    if (levels is None) or pd.isna(atr_val) or pd.isna(close):
        return False, np.nan
    band = float(mult) * float(atr_val)
    lo = min(levels['0.382'], levels['0.618'])
    hi = max(levels['0.382'], levels['0.618'])
    inside = (close >= lo - band) and (close <= hi + band)
    dist = min(abs(close - levels['0.382']),
               abs(close - levels['0.5']),
               abs(close - levels['0.618']))
    return bool(inside), float(dist)

# =========================
# Señales con confluencia
# =========================
def signals_with_confluence(ticker=TICKER, start=START, end=END,
                            cross_fast=MA1, cross_slow=MA2,
                            trend_ma=TREND_MA, confirm_ema=CONFIRM_EMA,
                            w=W_SWING, score_threshold=SCORE_THRESHOLD):
    df = download_prices(ticker, start, end, auto_adjust=True)
    if df.empty:
        return pd.DataFrame()

    # Indicadores
    df['MA_fast']   = sma(df['Close'], cross_fast)
    df['MA_slow']   = sma(df['Close'], cross_slow)
    df['MA_trend']  = sma(df['Close'], trend_ma)
    df['EMA_conf']  = ema(df['Close'], confirm_ema)
    df['ATR14']     = atr(df, 14)
    df['slope_tr']  = slope(df['MA_trend'], 5)

    # Cruces (Golden/Death entre MA1 y MA2)
    df['golden_cross'] = (df['MA_fast'].shift(1) <= df['MA_slow'].shift(1)) & (df['MA_fast'] > df['MA_slow'])
    df['death_cross']  = (df['MA_fast'].shift(1) >= df['MA_slow'].shift(1)) & (df['MA_fast'] < df['MA_slow'])

    # Pivotes para swings
    df = pivots(df.copy(), w=w)

    rows = []
    idx = df.index.to_list()
    for i, ts in enumerate(idx):
        row = df.loc[ts]
        if pd.isna(row['MA_trend']):
            continue

        # Tendencia por filtro (precio vs MA_trend y pendiente)
        trend = 'up' if (row['MA_trend'] < row['Close'] and row['slope_tr'] > 0) \
                 else ('down' if (row['MA_trend'] > row['Close'] and row['slope_tr'] < 0) else None)
        if not trend:
            continue

        # Niveles de Fib del último swing según tendencia
        sub = df.loc[:ts]
        lv = last_swing_levels(sub, trend)

        # Proximidad a zona Fib con buffer ATR
        hit, dist = fib_zone_hit(row['Close'], lv, row['ATR14'], mult=0.5)

        # Confirmación
        confirm = (row['Close'] > row['EMA_conf']) if trend=='up' else (row['Close'] < row['EMA_conf'])

        # Cruce reciente (últimas 5 velas)
        recent_cross = False
        for k in range(1,6):
            j = i-k
            if j<0: break
            if trend=='up' and bool(df['golden_cross'].iloc[j]):
                recent_cross = True; break
            if trend=='down' and bool(df['death_cross'].iloc[j]):
                recent_cross = True; break

        # Puntuación
        score = 2 + (3 if hit else 0) + (2 if confirm else 0) + 1 + (2 if recent_cross else 0)

        if score >= score_threshold:
            rows.append({
                'date': ts, 'ticker': ticker, 'trend': trend,
                'close': float(row['Close']), 'score': int(score),
                'recent_cross': bool(recent_cross), 'in_fib_zone': bool(hit),
                'fib_dist': (float(dist) if not pd.isna(dist) else np.nan),
                'ema_confirm': bool(confirm),
                'atr': (float(row['ATR14']) if not pd.isna(row['ATR14']) else np.nan),
                'levels': lv
            })

    out = pd.DataFrame(rows)
    return out.set_index('date') if not out.empty else out

# =========================
# Gráfico interactivo
# =========================
def plot_confluence_chart(ticker=TICKER, start=START, end=END,
                          cross_fast=MA1, cross_slow=MA2,
                          trend_ma=TREND_MA, confirm_ema=CONFIRM_EMA,
                          w=W_SWING, score_threshold=SCORE_THRESHOLD,
                          last_n=LAST_N, show_ext=SHOW_EXT):
    df = download_prices(ticker, start, end, auto_adjust=True)
    if df.empty or df['Close'].isna().all():
        print("Sin datos para graficar.")
        return go.Figure()

    # Indicadores
    df['MA_fast']   = sma(df['Close'], cross_fast)
    df['MA_slow']   = sma(df['Close'], cross_slow)
    df['MA_trend']  = sma(df['Close'], trend_ma)
    df['EMA_conf']  = ema(df['Close'], confirm_ema)
    h,l,c = df['High'], df['Low'], df['Close']
    tr = np.maximum(h - l, np.maximum((h - c.shift()).abs(), (l - c.shift()).abs()))
    df['ATR14']     = tr.rolling(14).mean()
    df['slope_tr']  = slope(df['MA_trend'], 5)

    # Cruces
    df['golden_cross'] = (df['MA_fast'].shift(1) <= df['MA_slow'].shift(1)) & (df['MA_fast'] > df['MA_slow'])
    df['death_cross']  = (df['MA_fast'].shift(1) >= df['MA_slow'].shift(1)) & (df['MA_fast'] < df['MA_slow'])

    # Swings
    df = pivots(df, w=w)

    # Señales (usa la misma lógica que arriba)
    sig = signals_with_confluence(ticker, start, end, cross_fast, cross_slow, trend_ma, confirm_ema, w, score_threshold)

    # Recorte
    df_plot = df.tail(last_n) if last_n else df
    idx0, idx1 = df_plot.index.min(), df_plot.index.max()

    # Tendencia actual en ventana
    last = df_plot.iloc[-1]
    trend = 'up' if (last['MA_trend'] < last['Close'] and last['slope_tr'] > 0) \
             else ('down' if (last['MA_trend'] > last['Close'] and last['slope_tr'] < 0) else None)

    # Fibs del último swing visible
    lv = last_swing_levels(df_plot, trend) if trend else None

    fig = go.Figure()

    # Velas
    fig.add_trace(go.Candlestick(
        x=df_plot.index, open=df_plot['Open'], high=df_plot['High'],
        low=df_plot['Low'], close=df_plot['Close'],
        name="OHLC", showlegend=False, opacity=0.95
    ))

    # Medias
    #fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['MA_fast'],  name=f"SMA#{cross_fast}", mode='lines'))
    fig.add_trace(go.Scatter(
            x=df_plot.index, y=df_plot['MA_fast'],
            name=f"SMA{cross_fast}", mode='lines',
            line=dict(color="#F2AF07", width=2)   # orange
        ))
    #fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['MA_slow'],  name=f"SMA{cross_slow}", mode='lines'))
    fig.add_trace(go.Scatter(
            x=df_plot.index, y=df_plot['MA_slow'],
            name=f"SMA{cross_slow}", mode='lines',
            line=dict(color="#0761F2", width=2)   # azul
        ))
    # Muestra el filtro de tendencia sólo si es distinto
    if trend_ma not in (cross_fast, cross_slow):
        fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['MA_trend'], name=f"SMA{trend_ma}", mode='lines'))

    # EMA de confirmación (si distinta, para evitar duplicar línea)
    if confirm_ema not in (cross_fast, cross_slow):
        fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['EMA_conf'], name=f"EMA{confirm_ema}", mode='lines'))


    # Cruces
    gc_idx = df_plot.index[df_plot['golden_cross']]
    dc_idx = df_plot.index[df_plot['death_cross']]
    if len(gc_idx):
        fig.add_trace(go.Scatter(
            x=gc_idx, y=df_plot.loc[gc_idx, 'Close'],
            mode='markers', name="Golden cross",
            marker=dict(symbol='triangle-up', size=10 ,color=UP_COLOR),
            hovertemplate="Golden cross<br>%{x}<br>Close=%{y:.2f}<extra></extra>"
        ))
    if len(dc_idx):
        fig.add_trace(go.Scatter(
            x=dc_idx, y=df_plot.loc[dc_idx, 'Close'],
            mode='markers', name="Death cross",
            marker=dict(symbol='triangle-down', size=10, color = DOWN_COLOR),
            hovertemplate="Death cross<br>%{x}<br>Close=%{y:.2f}<extra></extra>"
        ))

    # Señales aceptadas (score ≥ threshold) en la ventana
    if isinstance(sig, pd.DataFrame) and not sig.empty:
        sig_win = sig.loc[(sig.index >= idx0) & (sig.index <= idx1)]
        if not sig_win.empty:
           fig.add_trace(go.Scatter(
              x=sig_win.index, y=sig_win['close'],
              mode='markers', name=f"Señal (score≥{score_threshold})",
              marker=dict(symbol='star', size=12, color=SIG_COLOR, line=dict(width=0)),
              opacity=SIGNAL_OPACITY,
              visible=(True if SHOW_SIGNALS else 'legendonly')  # ocultas de inicio si False
          ))


    # Fibs 38.2–61.8 + 78.6 + extensiones
    if lv is not None:
        y382, y50, y618 = lv['0.382'], lv['0.5'], lv['0.618']
        y786 = lv['0.786']
        y1272 = lv['ext_1.272']; y1618 = lv['ext_1.618']

        # Zona 38.2–61.8
        y0, y1 = min(y382, y618), max(y382, y618)
        fig.add_shape(
            type="rect", x0=idx0, x1=idx1, y0=y0, y1=y1,
            fillcolor="rgba(255, 215, 0, 0.10)", line=dict(width=0), layer="below"
        )

        # Líneas guía
        def add_hline(y, name):
            fig.add_trace(go.Scatter(
                x=[idx0, idx1], y=[y, y], mode='lines', name=name,
                line=dict(dash='dash'), hoverinfo='skip'
            ))
        add_hline(y382, "Fib 38.2%")
        add_hline(y50,  "Fib 50%")
        add_hline(y618, "Fib 61.8%")
        add_hline(y786, "Fib 78.6%")
        if show_ext:
            add_hline(y1272, "Ext 1.272")
            add_hline(y1618, "Ext 1.618")

    fig.update_layout(
            title=dict(
                text=f"{ticker} · MA{cross_fast}/{cross_slow} + Fib (últimas {len(df_plot)} velas)",
                x=0.5, y=0.98, xanchor='center', yanchor='top'
            ),
            xaxis_title="Fecha", yaxis_title="Precio",
            xaxis_rangeslider_visible=False, hovermode="x unified",
            legend=dict(
                orientation='h',
                yanchor='top', y=-0.18,        # <--- fuera del área, abajo
                xanchor='center', x=0.5,
                bgcolor='rgba(255,255,255,0.85)',
                bordercolor='rgba(0,0,0,0.1)', borderwidth=1,
                groupclick='togglegroup'       # <--- clic para ocultar/mostrar grupos
            ),
            margin=dict(t=90, b=120, l=60, r=30)
    )
    return fig



In [6]:
# =========================
# Ejecutar
# =========================
signals = signals_with_confluence(
    ticker=TICKER, start=START, end=END,
    cross_fast=MA1, cross_slow=MA2,
    trend_ma=TREND_MA, confirm_ema=CONFIRM_EMA,
    w=W_SWING, score_threshold=SCORE_THRESHOLD
)
#display(signals.tail(10))

fig = plot_confluence_chart(
    ticker=TICKER, start=START, end=END,
    cross_fast=MA1, cross_slow=MA2,
    trend_ma=TREND_MA, confirm_ema=CONFIRM_EMA,
    w=W_SWING, score_threshold=SCORE_THRESHOLD,
    last_n=LAST_N, show_ext=SHOW_EXT
)
fig.show()